# Day 2 

## Module 4

The amount of data generated each day has been growing exponentially over the past years. This may result in the generation of a large number of input features, a subset of which may be highly correlated, repetitive, not very informative or even related to the particular study. Datasets with a large number of features are commonly referred to as high-dimensional datasets.

In this module, we will investigate Principal Component Analysis (PCA) as a technique for dimensionality reduction, data compression and feature extraction. The PCA algorithm reduces the initial number of possibly correlated variables into a new lower number of uncorrelated (orthogonal) variables, known as the Principal Components (PCs). PCA is commonly applied prior to unsupervised and supervised machine learning models to reduce the number of features used in the analysis, thereby reducing the likelihood of error.


### Learning Activity - Loading Libraries

We will start once more by loading the required Python libraries.

In [ ]:
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.plotly as py

# Module 4
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

# Module 5
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import metrics

from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

%matplotlib inline
init_notebook_mode()

print("libraries all imported, ready to go")

### Learning Activity: Reading in the data

As a first step, we will need to import the data from the "`retail_ml_dataset.csv`" data file that we constructed and exported on Day 1 (or the backup file "`retail_ml_dataset_backup.csv`" that we have provided) into the variable **_X_** using the `read_csv()` function from `pandas` (`pd`). We also want to define the column that we are going to use as the row labels of the DataFrame; in this case, *'CustomerID'*. Once loaded, we can apply once more the `head()` function to preview the first 5 rows of our DataFrame. 

In [ ]:
# Import the data from the retail_ml_dataset.csv, set the index column and explore the first few rows

X = pd.read_csv('data/retail_ml_dataset.csv', index_col='CustomerID')
X.head()

Remember, it is **always** a good practice to check the dimensionality of the imported data using the `shape` command prior to constructing any classification model to make sure you have correctly imported all the data (e.g. one common mistake is to get the separator wrong and end up with only one column). 

In [ ]:
# Check the dimensionality of X

print(X.shape)

### Test Activity - Reading in the classes 

Similarly, try to import the customer classes from the provided "`customer_classes.csv`" into the variable **_y_**. Remember to also define the column that will be used as the row labels of the DataFrame as in the previous step. 

In this case, the class **_y_** contains two classes (binary case) - "yes" vs. "no" - that represent the returning and non-returning customers respectively.

In [ ]:
# Import the data from the customer_classes.csv, set the index column and explore the first few rows

y = pd.read_csv('data/customer_classes.csv', index_col='CustomerID')
y.head()

In [ ]:
# Check the dimensionality of y

print(y.shape)

- *** What do you observe with regards to the dimensionality of X and y?***

### Learning Activity - Investigate the y frequencies


An important aspect to understand before applying any classification algorithm is how the output labels are distributed. Are they evenly distributed or not? Imbalances in distribution of labels can often lead to poor classification results for the minority class even if the classification results for the majority class are very good. 

In [ ]:
# Calculate and print the y frequencies

yFreq = scipy.stats.itemfreq(y)
yFreq = pd.DataFrame(yFreq, columns=['class','freq'])
print(yFreq)

Visualising the class frequencies is a good way to get a feel for how the data is distributed. As a simple example, try plotting the frequencies of the class labels (held in _yFreq_), "yes" and "no", and see how they are distributed using a barplot from `seaborn`:

In [ ]:
# Display the y frequencies in a barplot

ax = sns.barplot(x="class", y="freq", data=yFreq)
ax.set(xlabel="Class", ylabel="Frequency")
plt.show()

### Learning Activity: Encode categorical values

In our current dataset, you can see that the **_y_** values are categorical (i.e. they only take one of a discrete set of values) and have a non-numeric representation, "yes" vs. "no". This can be problematic for scikit-learn and plotting functions in Python, since they assume numerical values, so we need to map the text categories to numerical representations using `LabelEncoder()`  and the `fit_transform()` function from the `preprocessing` module:

In [ ]:
# Convert the categorical values into numbers using the label encoder

# Initialise a LabelEncoder object
le = LabelEncoder()

# Fit label encoder, return encoded labels and assign back to the class column of y
y['class'] = le.fit_transform(y['class'])

# Show the results
y.head()

We can observe that the categorical values have been mapped to numeric values based on their alphabetic order ('no' to 0 and 'yes' to 1). More information on `LabelEncoder()`, its parameters and how to use it can be found at http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

### Learning Activity - Joining with class

A very useful feature of `pandas` is its `join()` function, which allows combining tables based on one column shared between tables. Here we use `join()` to combine the input features and information on whether customers return (associated classes).

You will join the class labels with the dataset by a shared index. **`CustomerID`** is the obvious choice here, as it is the only column shared between the two DataFrames.

In [ ]:
# Join the X and y DataFrames based on the common CustomerIDs

customers = X.join(y)
customers.head()

Again, it is a good sanity check to investigate the dimensionality of our newly created DataFrame

In [ ]:
# Check the dimensionality

print (customers.shape)

- *** What do you observe with regards to the dimensionality of X and y?***

### Learning Activity - Investigate the relationship between input features

Visualisation is an integral part of Data Science. Exploratory data analysis (EDA) is the field dealing with the analysis of data sets as a means of summarising their main characteristics, most often using visual methods.

One of the most powerful tools commonly used as part of EDA is the scatter plot. This step helps visualise the relationship in-between two input features, indicates the degree of correlation plus helps reveal any patterns and trends in the data, and also gives you a first indication of the ML model that could be applied and its complexity (linear vs. non-linear). 

In [ ]:
# Create a scatter plot of the first two features 'balance' vs. 'total_spent'

ax = sns.lmplot('balance', 'max_spent', data=customers, fit_reg=False)
plt.show()

This is a great start, but we may want to create a more informative scatterplot by relating associations between the input points to their _y_ labels (classes) by making the colour and/or marker (shape) of the points dependent on the corresponding _y_ value. At this stage, let’s plot again these two variables, but also make use of the class information:

In [ ]:
# Create an enhanced scatter plot of the features 'balance' vs. 'total_spent'

ax = sns.lmplot('balance', 'max_spent', data=customers, hue='class', markers=['o', 'x'], 
               palette=['blue', 'orange'], fit_reg=False)

plt.show()

### Test Activity - Create an enhanced scatterplot of different features

Try once more creating an enhanced scatterplot as before, but this time using the features 'balance' and 'total_spent'. What do you observe? 

In [ ]:
# Create an enhanced scatter plot of the features 'balance' vs. 'total_spent'

ax = sns.lmplot('balance', 'total_spent', data=customers, hue='class', markers=['o', 'x'], 
               palette=['blue', 'orange'], fit_reg=False)

plt.show()

### Bonus Activity - Try different combinations of the input features in a grid/scatterplot matrix

A scatterplot matrix shows a grid of scatterplots where each attribute is plotted against all other attributes. You can find further information on how to create a scatterplot matrix with seaborn using the pairplot() function at https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.pairplot.html. Try to colour once more the observations based on their associated class. 

In [ ]:
# Create a scatterplot matrix 

ax = sns.pairplot(customers, hue='class', markers=['o', 'x'], palette=['blue', 'orange'])
plt.show()

### Learning Activity - Create a correlation matrix and heatmap of correlations between the input features of X

It is often of great interest to investigate whether any of the variables in a multivariate dataset are significantly correlated. As we previously demonstrated, the different features (variables) in **_X_** are not completely independent from each other. To quickly identify which features are related and to
what degree, it is useful to to calculate a correlation matrix that shows the correlation coefficient for each pair of variables. You can do this by using the `corr()` function from the `pandas` library:

In [ ]:
# Calculate the correlation coefficients

corrmat = X.corr()

To search for linear relationships between features across all pairs of features, you can use a heatmap
of correlations, which is simply a matrix of subplots whose colours represent the
degree of the correlations:

In [ ]:
# Create a heatmap of the correlation coefficients

f, ax = plt.subplots(figsize=(12, 9))
ax = sns.heatmap(corrmat, center=0, linewidths=0.5, cmap="RdBu", vmin=-1, vmax=1, annot=True)
plt.show()

### Learning Activity: Mapping Categorical Features

As previously mentioned, most machine learning libraries and tools will only accept numerical values as their input. In the case where we have categorical features present, we need to represent them as numerical values. Unlike the label encoding we demonstrated previously for the class vector, when dealing with categorical input features, typically one converts each categorical feature using “one-hot encoding”. The input in one-hot encoding is the vector of discrete categorical values, and the output will be a sparse matrix where each column corresponds to one possible value of one feature.

In our example, the feature Country is a categorical feature with values such as ["Finland", "Norway", "Spain"] etc. Such features can be easily mapped to dummy variables which could be expressed as [0, 1, 2]. Can you spot any problem though with this approach? Even though the country values do not come in any particular order, a machine learning algorithm will now assume that "Spain" is larger than "Norway", and "Norway" is larger than "Finland", and so on. Although this assumption is incorrect, the algorithm could still produce useful results. However, those results would not be optimal.  

The correct approach in this case is to apply one-hot encoding. This estimator transforms each unique categorical value of a single input categorical feature to a new dummy **feature**. So, for our 11 unique country values (remember the learning activity from day 1), we will end up with 10 new dummy features after one-hot-encoding. 

There are plenty of libraries and functions that are used for one-hot encoding. In this example, we will use the `get_dummies()` function from `pandas`: 

In [ ]:
# Apply one-hot encoding to the categorical feature Country

countries = customers['Country']
countries = pd.get_dummies(countries)
countries.head()

At this stage, we want to update in our *customers* and *X* Dataframes the categorical feature "Country" with the result of one-hot encoding. Remember, that are going to remove a single feature, and add 11 new dummy variables created by one-hot encoding. 

In [ ]:
# Remove and replace the Country values from customers with the result of one-hot encoding

customers = customers.drop('Country', 1)
customers = countries.join(customers)
customers.head()

In [ ]:
# Remove and replace the Country values from X with the result of one-hot encoding

X = X.drop('Country', 1)
X = countries.join(X)
X.head()

In [ ]:
# Check the dimensionality after one-hot encoding

print(customers.shape)

## Principal Component Analysis 

Principal Component Analysis (PCA) is the most commonly used technique for dimensionality reduction, data compression and feature extraction. The PCA algorithm reduces the initial number of possibly correlated variables into a new lower number of uncorrelated variables, more specifically in a set of successive orthogonal components that explain a maximum amount of the variance, known as the Principal Components (PCs). 

### Learning Activity - Apply PCA in the input data using scikit-learn

In scikit-learn, PCA is implemented as a transformer object that learns _n_ components in its `fit()` method, and can be used on new data to project it on these components. More information on how to use the `pca()` function and its parameters can be found at http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html


In [ ]:
# Initialise the PCA object and create an index for each Principal Component

pca = PCA()
ind = ['PC'+str(i+1) for i in range(X.shape[1])]

The values of the Principal Components (Xscores) can be computed by the `fit_transform()` (alternatively, `fit()` followed by `transform()`) function. This function returns a matrix with the principal components, where the first column in the matrix contains the first principal component, the second column the second component, and so on.

In [ ]:
# Create the PCA scores matrix and check the dimensionality of the PCA scores

Xscores = pca.fit_transform(X)
Xscores = pd.DataFrame(Xscores, columns = ind, index = X.index)

print(Xscores.shape)

The loadings for the principal components are stored in a named element *components_*. This contains a matrix with the loadings of each principal component, where the first column in the matrix contains the loadings for the first principal component, the second column contains the loadings for the second principal component, and so on. 

In [ ]:
# Create the PCA loadings matrix and check the dimensionality of the PCA loadings

loadings = pca.components_
loadings = pd.DataFrame(loadings, columns=X.columns, index = ind)

print(loadings.shape)

### Learning Activity: Calculate and plot the explained and cumulative variance 

But how much information have we lost? We can figure this out by looking at the explained and cumulative variance. The explained variance gives us the proportion of variance explained by each successive Principal Component. The cumulative variance  is obtained by adding the successive proportions of explained variance to obtain the total sum.

In [ ]:
# Calculate the explained and cumulative variance

# Calculate the explained variance
exp_var = [i*100 for i in pca.explained_variance_ratio_]

# Calculate the cumulative variance
cum_var = np.cumsum(pca.explained_variance_ratio_*100)

# Combine both in a data frame
pca_var = pd.DataFrame(data={'exp_var': exp_var, 'cum_var': cum_var}, index=ind)
pca_var

We can also plot the explained variance using a barplot with seaborn:

In [ ]:
# Plot the explained variance per PC using a barplot

fig = plt.figure(figsize=(10,7))

ax = sns.barplot(x=pca_var.index, y='exp_var', data=pca_var)
ax.set(xlabel='Principal Components', ylabel='Explained Variance')
plt.show()

### Bonus Activity - Plot the cumulative variance

Plot the cumulative variance using a line plot, screeplot or barplot:

In [ ]:
# Plot the cumulative variance per PC using a barplot

fig = plt.figure(figsize=(10,7))
g = sns.barplot(x=pca_var.index, y='cum_var', data=pca_var)
g.set(xlabel='Principal Components', ylabel='Cumulative Variance')
plt.show()

### Learning Activity - Plot the PCA loadings using a heatmap

What the PCA does is construct new variables (or principal components) that explain most of the variance or scatter of the original dataset. Each component is a linear combination of all the variables and is perpendicular to every other component. Each variable in each component is multiplied by set of factors, the loading factors, which transforms the original data into this new component space. These loading factors are constrained so that the square of the sum is equal to 1, hence they can serve as weights to see which parameters are most important for a particular principal component. Let's look at that in more detail with some figures.

In [ ]:
f, ax = plt.subplots(figsize=(20, 16))

ax = sns.heatmap(loadings.transpose(), center=0, linewidths=0.5, cmap="RdBu", vmin=-1, vmax=1, annot=True)
ax.set_xticklabels(ax.xaxis.get_majorticklabels(), rotation=0)
ax.set_yticklabels(ax.yaxis.get_majorticklabels(), rotation=0, fontsize=12)
plt.show()

We can use this heatmap to see how the different features load onto the different components. For example, PC1 seems to be concerned mainly with volume of purchase (hence the heavy loadings of the item quantity and spending amounts), while PC 3 seems to be mainly concerned with the total refunded. In real life scenarios, these components can often be attributed to higher level concepts with respect to the features (e.g. length, breadth, height might form a "size" component) or domain-specific constructs. 

### Learning Activity - Create a scatterplot of the first two PCs

It is quite useful to create a scatterplot of the first two principal components (Xscores of PC1 and PC2), and colour the observations based on their associated class labels:

In [ ]:
# Create an enhanced scatterplot of the first two Principal Components
# Remember, we are using Xscores (!) rather than X as input at this stage

ax = sns.lmplot('PC1','PC2', data= Xscores.join(y), hue='class', markers=['o', 'x'], 
                palette=['blue', 'orange'], fit_reg=False)

plt.show()

The scatterplot shows PC1 (1st Principal Component) on the x-axis, and PC2 (2nd Principal Component) on the y-axis. Based on this plot, we can see that the observations of the two classes are highly overlapping in 2D space, therefore, it may be interesting to investigate the first 3 PCs using a 3D scatterplot. 

### Bonus Activity: Create a 3D PCA scores plot using seaborn or Plotly

We may want to investigate simultaneously the first three Principal Components (PCs) using seaborn or Plotly. *Hint* you may need to use a for loop to iterate through each of the available classes while building the scatterplot. 

In [ ]:
# Create a 3D scatterplot of the first three Principal Components 

desc = dict(
    classes = [1, 0],
    colors  = ["orange", "blue"],
    labels  = ["Returning customers", "Non-returning customers"],
    symbols = ["x", "circle"]
)

data = [
    Scatter3d(
        x = Xscores['PC1'].loc[y['class'] == desc["classes"][i]],
        y = Xscores['PC2'].loc[y['class'] == desc["classes"][i]],
        z = Xscores['PC3'].loc[y['class'] == desc["classes"][i]],
        name = desc["labels"][i],
        mode = "markers",
        marker = dict(
            size = 2.5,
            symbol = desc["symbols"][i],
            color  = desc["colors"][i]
        )
    ) for i in range(len(desc["classes"]))
]

layout = Layout(
    scene=Scene(
        xaxis=XAxis(title='PC1', titlefont=dict(size=11)),
        yaxis=YAxis(title='PC2', titlefont=dict(size=11)),
        zaxis=ZAxis(title='PC3', titlefont=dict(size=11))
    ),
    margin=Margin(l=80, r=80, b=0, t=0, pad=0, autoexpand=True),
    height= 600,
)

fig = dict(data = data, layout = layout)

iplot(fig)

## Module 5

This module will focus on introducing, building and optimising a classification model, the K Nearest Neighbor (KNN) classifier. The principle behind KNN is very simple. Given a dataset where you know the class labels, when a new point is introduced, you want to find a particular number of points in the data closest in distance to the new point. These are called the “nearest neighbours”.  You then use the labels associated with these nearest neighbour points (which may or may not be different from each other) to predict the label of the new point using a majority vote.

### Learning Activity: Convert DataFrames to numpy arrays

In order to feed the data into our classification models in scikit-learn, the imported DataFrames need to be converted into `numpy` arrays. For more information on numpy arrays, read the documentation at http://scipy-lectures.github.io/intro/numpy/array_object.html. 


In [ ]:
# Convert to numpy array and check the dimensionality

customers = np.array(customers)
X = customers[:,:-1].astype(float)
y = customers[:,-1]

print (X.shape)
print (y.shape)

### Learning Activity - Split the data into training and test sets (raw data prior to PCA)

Training and testing a classification model on the same dataset is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data (poor generalisation). To use different datasets for training and testing, we need to split the wine dataset into two disjoint sets: train and test (**Holdout method**) using the `train_test_split` function. <br/> 

In [ ]:
# Split the raw data into training and test sets

XTrain, XTest, yTrain, yTest = train_test_split(X, y, random_state=1)

# Print the dimensionality of the individual splits

print ("XTrain dimensions: ", XTrain.shape)
print ("yTrain dimensions: ", yTrain.shape)
print ("XTest dimensions: ", XTest.shape)
print ("yTest dimensions: ", yTest.shape)

### Learning Activity - Train, optimise and test a KNN algorithm with scikit-learn

To build KNN models using scikit-learn, you will be using the `KNeighborsClassifier` object, which allows you to set the value of K using the `n_neighbors` parameter (http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). For every classification model built with scikit-learn, we will follow four main steps: 
1) **Building** the classification model (using either default, pre-defined or optimised parameters), 
2) **Training** the model, 
3) **Testing** the model, and 
4) **Performance evaluation** using various metrics. <br/> <br/>

The optimal choice for the value K is highly data-dependent: in general a larger K suppresses the effects of noise, but makes the classification boundaries less distinct. Rather than trying one-by-one predefined values of K, we can automate this process. The scikit-learn library provides the grid search function `GridSearchCV` (http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html), which allows us to exhaustively search for the optimum combination of parameters by evaluating models trained with a particular algorithm with all provided parameter combinations. Further details and examples on grid search with scikit-learn can be found at http://scikit-learn.org/stable/modules/grid_search.html. You can use the `GridSearchCV` function with the validation technique of your choice (in this example, 10-fold cross-validation has been applied) to search for a parametisation of the KNN algorithm that gives a more optimal model:

In [ ]:
# Grid search with 10-fold cross-validation using a dictionary of parameters

# Create the dictionary of given parameters
n_neighbors = np.arange(1, 101, 2)  
parameters  = [{'n_neighbors': n_neighbors}] 

# Optimise and build the model with GridSearchCV
gridCV = GridSearchCV(KNeighborsClassifier(), parameters, cv=10)
gridCV.fit(XTrain, yTrain) 

# Report the optimal parameters
bestNeighbors = gridCV.best_params_['n_neighbors']

print("Best parameters: n_neighbors=", bestNeighbors)

When evaluating the resulting model it is important to do it on held-out samples that were not seen during the grid search process (XTest). <Br/>
So, we are testing our independent XTest dataset using the optimised model:

In [ ]:
# Build the classifier using the optimal parameters detected by grid search

knn = KNeighborsClassifier(n_neighbors=bestNeighbors)

# Fit to the training set 
knn.fit(XTrain, yTrain)

# Predict the test data
yPredKnn = knn.predict(XTest)

# Report the final overall accuracy
print ("Overall Accuracy:", round(metrics.accuracy_score(yTest, yPredKnn), 2))

### Test Activity - Split the PCA scores and class labels into training and test sets


At this stage, we want to repeat the whole model-building process, starting by applying holdout like before, but this time apply it to the data generated by PCA (PCA scores) and their associated class labels. As before, you need to use the `train_test_split()` function, and remember to keep the same seed (`random_state=1`) for direct and fair comparison! You may need to select the number of PCs from your Xscores that you will feed into your model, or create a for loop to try and test different values. 

In [ ]:
# Split the results of PC scores into training and test sets

XTrain, XTest, yTrain, yTest = train_test_split(Xscores.values[:,0:3], y, random_state=1)

# Print the dimensionality of the individual splits

print ("XTrain dimensions: ", XTrain.shape)
print ("yTrain dimensions: ", yTrain.shape)
print ("XTest dimensions: ", XTest.shape)
print ("yTest dimensions: ", yTest.shape)

### Learning Activity - Train, optimise and test a KNN algorithm with scikit-learn

Repeat the process of training and optimising a KNN classifier as before, but this time, apply it to your newly created train and test data (after PCA). 

In [ ]:
# Grid search with 10-fold cross-validation using a dictionary of parameters

# Create the dictionary of given parameters
n_neighbors = np.arange(1, 101, 2)  
parameters  = [{'n_neighbors': n_neighbors}] 

# Optimise and build the model with GridSearchCV
gridCV = GridSearchCV(KNeighborsClassifier(), parameters, cv=10)
gridCV.fit(XTrain, yTrain) 

# Report the optimal parameters
bestNeighbors = gridCV.best_params_['n_neighbors']

print("Best parameters: n_neighbors=", bestNeighbors)

In [ ]:
# Build the classifier using the optimal parameters detected by randomised search

knn = KNeighborsClassifier(n_neighbors=bestNeighbors)
knn.fit(XTrain, yTrain)
yPredKnn = knn.predict(XTest)

print ("Overall Accuracy:", round(metrics.accuracy_score(yTest, yPredKnn), 2))